# Ex2 - Getting and Knowing your Data

This time we are going to pull data directly from the internet.
Special thanks to: https://github.com/justmarkham for sharing the dataset and materials.

### Step 1. Import the necessary libraries

In [68]:
import pandas as pd

### Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv). 

In [69]:
url = 'https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv'
dataset = pd.read_csv(url, sep='\t')
#为何sep='\t'，因为读取的是tsv(tab-separated values)，所以sep(seperator)要用这个；这个值默认是','
#因为我们通常读取的是csv(comma-separated values)

### Step 3. Assign it to a variable called chipo.

In [70]:
chipo = dataset

### Step 4. See the first 10 entries

In [71]:
chipo.head(10)

,order_id,quantity,item_name,choice_description,item_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,$2.39
1,1,1,Izze,[Clementine],$3.39
2,1,1,Nantucket Nectar,[Apple],$3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,$2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",$16.98
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",$10.98
6,3,1,Side of Chips,NaN,$1.69
7,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables...",$11.75
8,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Ch...",$9.25
9,5,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Pinto...",$9.25


### Step 5. What is the number of observations in the dataset?

In [72]:
# Solution 1
chipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4622 entries, 0 to 4621
Data columns (total 5 columns):
order_id              4622 non-null int64
quantity              4622 non-null int64
item_name             4622 non-null object
choice_description    3376 non-null object
item_price            4622 non-null object
dtypes: int64(2), object(3)
memory usage: 180.6+ KB


In [73]:
# Solution 2
chipo.shape

(4622, 5)

### Step 6. What is the number of columns in the dataset?

In [74]:
chipo.shape[1]

5

### Step 7. Print the name of all the columns.

In [75]:
print(chipo.columns)

Index([u'order_id', u'quantity', u'item_name', u'choice_description',
       u'item_price'],
      dtype='object')


### Step 8. How is the dataset indexed?

In [76]:
chipo.index

RangeIndex(start=0, stop=4622, step=1)

### Step 9. Which was the most-ordered item? 

#### My answer(wrong)
我这个只考虑到单次购买产品最大的次数，而不是某个产品累计的出售次数

In [77]:
mask = chipo['quantity'] == chipo['quantity'].max()
most_order_item = chipo[mask]
most_order_item

,order_id,quantity,item_name,choice_description,item_price
3598,1443,15,Chips and Fresh Tomato Salsa,NaN,$44.25


#### Correct Answer

In [78]:
chipo_groupby_itemname = chipo.groupby('item_name')
chipo_groupby_itemname = chipo_groupby_itemname.sum()
chipo_groupby_itemname.pop('order_id') #order_id只是个标记，在这里累加没用
chipo_groupby_itemname.head()
#这里通过.groupby把所有的产品弄出来

,quantity
item_name,
6 Pack Soft Drink,55
Barbacoa Bowl,66
Barbacoa Burrito,91
Barbacoa Crispy Tacos,12
Barbacoa Salad Bowl,10


In [79]:
chipo_groupby_itemname = chipo_groupby_itemname.sort_values(['quantity'], ascending=False)
chipo_groupby_itemname.head(1)

,quantity
item_name,
Chicken Bowl,761


### Step 10. For the most-ordered item, how many items were ordered?

#### My Answer(wrong)

In [80]:
most_order_item['quantity']

3598    15
Name: quantity, dtype: int64

#### Correcr Answer

In [81]:
chipo_groupby_itemname.iloc[0,0]

761

### Step 11. What was the most ordered item in the choice_description column?

#### Wrong Answer

In [82]:
most_order_item['choice_description']

3598    NaN
Name: choice_description, dtype: object

#### Correct Answer

In [83]:
c = chipo.groupby('choice_description').sum()
c = c.sort_values(['quantity'], ascending=False)
c.head(1)
# Diet Coke 159

,order_id,quantity
choice_description,,
[Diet Coke],123455,159


### Step 12. How many items were orderd in total?

In [84]:
chipo['quantity'].sum()

4972

### Step 13. Turn the item price into a float

#### Step 13.a. Check the item price type

In [85]:
chipo['item_price'].dtype

dtype('O')

#### Step 13.b.1 Create a lambda function and change the type of item price
我的做法

In [86]:
from re import sub
from decimal import Decimal
format = lambda x: Decimal(sub(r'[^\d.]', '', x))
chipo['item_price'] = chipo['item_price'].map(format)
chipo['item_price'] = chipo['item_price'].astype(float)

#### Step 13.b.2 其它做法

In [ ]:
dollarizer = lambda x: float(x[1:-1])
chipo.item_price = chipo.item_price.apply(dollarizer)
#如果要运行这一个cell的话，不能运行上面那个cell，因为那里的item_price已经是float了

#### Step 13.c. Check the item price type

In [88]:
chipo['item_price'].dtype

dtype('float64')

### Step 14. How much was the revenue for the period in the dataset?

#### My answer(wrong) 因为忽略了商品数量

In [89]:
revenue = chipo['item_price'].sum()
print(revenue)

34500.16


#### Correct Answer

In [90]:
chipo['revenue'] = chipo['quantity'] * chipo['item_price']
revenue = chipo['revenue'].sum()

In [91]:
print(revenue)

39237.02


### Step 15. How many orders were made in the period?

In [92]:
orders_sum = len(chipo['order_id'].unique())
print(orders_sum)

1834


### Step 16. What is the average revenue amount per order?

In [93]:
# Solution 1
average_revenue = revenue / orders_sum
print(average_revenue)

21.39423118865867


In [94]:
# Solution 2
order_grouped = chipo.groupby(by=['order_id']).sum()
order_grouped.mean()['revenue']

21.394231188658654

### Step 17. How many different items are sold?

In [95]:
items_count = len(chipo['item_name'].unique())
print(items_count)

50
